In [4]:
import pandas as pd
import numpy as np
import nltk
import json
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold
#nltk.download('all', force=True)
#nltk.data.path.append("/Users/laurenceliao/nltk_data")

In [6]:
tech_data = pd.read_csv("tech_jobs_trainingset.csv")

tech_data

,,jobdescription,jobtitle,skills,tag,tech,Unnamed: 6,Unnamed: 7
0,https://www.dice.com/jobs/detail/Business-Solu...,"GalaxE.SolutionsEvery day, our solutions affec...",Business Solutions Architect,"Enterprise Solutions Architecture, business in...",architect,yes,NaN,bi
1,https://www.dice.com/jobs/detail/SAP-FICO-Arch...,We are looking for a Senior SAP FICO Architect...,SAP FICO Architect,"FICO, AR, AP, Asset Management, HAHA",architect,yes,NaN,computer_vision
2,https://www.dice.com/jobs/detail/Java-Architec...,Our client in the healthcare software industry...,"Java Architect - Denver, CO - Fulltime",(See Job Description),architect,yes,NaN,NaN
3,https://www.dice.com/jobs/detail/Sr.-Software-...,PLEASE JOIN OUR TALENT NETWORK: http://bit.ly/...,Sr. Software Storage Engineer,"C and C++, Linux, UNIX",architect,yes,NaN,NaN
4,https://www.dice.com/jobs/detail/C%252B%252B-S...,Our client is seeking a Software Developer to ...,C++ Software Engineer,Bachelor's degree with 5 years of experience d...,architect,yes,NaN,NaN
...,...,...,...,...,...,...,...,...
1412,NaN,Frontend Web Developers design and implement u...,Web Developer,"HTML, CSS, JavaScript Frontend frameworks (e.g...",web_dev,yes,NaN,NaN
1413,NaN,Backend Web Developers build and maintain the ...,Web Developer,"Server-side programming (e.g., Node.js, Python...",web_dev,yes,NaN,NaN
1414,NaN,Frontend Web Designers create the visual eleme...,Web Designer,Frontend web development HTML/CSS Responsive d...,web_dev,yes,NaN,NaN
1415,NaN,Frontend Web Developers design and implement u...,Web Developer,"HTML, CSS, JavaScript Frontend frameworks (e.g...",web_dev,yes,NaN,NaN


In [7]:
# Data Creation
tech_training = pd.DataFrame(tech_data[['jobtitle', 'jobdescription', 'skills', 'tech', 'tag']])

tech_training = tech_training.dropna()
tech_training = tech_training.reindex()
tech_training

,jobtitle,jobdescription,skills,tech,tag
0,Business Solutions Architect,"GalaxE.SolutionsEvery day, our solutions affec...","Enterprise Solutions Architecture, business in...",yes,architect
1,SAP FICO Architect,We are looking for a Senior SAP FICO Architect...,"FICO, AR, AP, Asset Management, HAHA",yes,architect
2,"Java Architect - Denver, CO - Fulltime",Our client in the healthcare software industry...,(See Job Description),yes,architect
3,Sr. Software Storage Engineer,PLEASE JOIN OUR TALENT NETWORK: http://bit.ly/...,"C and C++, Linux, UNIX",yes,architect
4,C++ Software Engineer,Our client is seeking a Software Developer to ...,Bachelor's degree with 5 years of experience d...,yes,architect
...,...,...,...,...,...
1412,Web Developer,Frontend Web Developers design and implement u...,"HTML, CSS, JavaScript Frontend frameworks (e.g...",yes,web_dev
1413,Web Developer,Backend Web Developers build and maintain the ...,"Server-side programming (e.g., Node.js, Python...",yes,web_dev
1414,Web Designer,Frontend Web Designers create the visual eleme...,Frontend web development HTML/CSS Responsive d...,yes,web_dev
1415,Web Developer,Frontend Web Developers design and implement u...,"HTML, CSS, JavaScript Frontend frameworks (e.g...",yes,web_dev


In [8]:
# Data Preprocessing
tech_training = tech_training.applymap(lambda x: x.lower() if isinstance(x, str) else x)
tech_training

old_word = r'\bit\b'

# This is to avoid the stopwords function from removing It since it reads it as "it"
tech_training['tag'] = tech_training['tag'].str.replace(old_word, 'information_technology', regex=True)
tech_training['tech'] = tech_training['tech'].str.replace(r'\bno\b', 'unrelated', regex=True)
tech_training['tech'] = tech_training['tech'].str.replace(r'\byes\b', 'tech', regex=True)




In [9]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

# Removes special characters
def remove_special_characters(text):
    pattern = r'[^a-zA-Z\s]'  
    cleaned_text = re.sub(pattern, '', text)
    
    return cleaned_text

def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# remove whitespace
def remove_whitespace(text):
    return " ".join(text.split())


# remove keyfault stopwords

def remove_stopwords(text):
    stop_words = set(stopwords.words("english"))
    word_tokens = word_tokenize(text)
    filtered_text = [word for word in word_tokens if word not in stop_words]
    return filtered_text

# Stemming 

stemmer = PorterStemmer()

def stem_words(text):
    word_tokens = word_tokenize(text)
    stems = [stemmer.stem(word) for word in word_tokens]
    return stems

# Lemmatization
lemmatizer = WordNetLemmatizer

def lemma_words(text):
    word_tokens = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word) for word in text]
    return lemmatized_words
    

tech_training = tech_training.applymap(lambda x: remove_special_characters(x) if isinstance(x, str) else x)
tech_training = tech_training.applymap(lambda x: remove_punctuation (x) if isinstance(x, str) else x)
tech_training = tech_training.applymap(lambda x: remove_whitespace(x) if isinstance(x, str) else x)
tech_training = tech_training.applymap(lambda x: remove_stopwords(x) if isinstance(x, str) else x)
tech_training = tech_training.applymap(lambda x: stem_words(x) if isinstance(x, str) else x)
tech_training = tech_training.applymap(lambda x: lemma_words(x) if isinstance(x, str) else x)





In [135]:

from sklearn.preprocessing import LabelEncoder
# Combining both job title and job description
#tech_training['combined'] = tech_training['jobtitle'] + tech_training['jobdescription']
tech_training['combined'] = tech_training['jobtitle'] + tech_training['jobdescription'] + tech_training['skills'] 
X = tech_training['combined'].apply(lambda x:x[0])




y = tech_training['tag'].apply(lambda x: x[0])


# Assuming 'y' contains your categorical labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Train Test Split (80% training data, 20% testing data)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)



In [145]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
X_train_Tfidf_df = tfidf_vectorizer.fit_transform(X_train).toarray()
X_train_Tfidf_df = pd.DataFrame(X_train_Tfidf_df)

X_test_Tfidf_df = tfidf_vectorizer.transform(X_test).toarray()
X_test_Tfidf_df = pd.DataFrame(X_test_Tfidf_df)

In [13]:
type(y)

numpy.ndarray

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
# Testing Stratified K Fold instead of K test split
tech_training['combined'] = tech_training['jobtitle'] + tech_training['skills'] + tech_training['jobdescription'] 
X = tech_training['combined'].apply(lambda x:x[0])


y = tech_training['tag'].apply(lambda x: x[0])
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)




skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Initialize 
    tfidf_vectorizer = TfidfVectorizer()

    # Fit transform the vectorizer

    X_train_Tfidf_df = tfidf_vectorizer.fit_transform(X_train).toarray()


    X_test_Tfidf_df = tfidf_vectorizer.transform(X_test).toarray()


    #tfidf_vectorizer.get_feature_names_out()
    rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)  # You can adjust parameters like n_estimators

# Train the classifier
    rf_classifier.fit(X_train_Tfidf_df, y_train)

# Predict on the test data
    y_pred_rf = rf_classifier.predict(X_test_Tfidf_df)

# Evaluate the classifier
    accuracy_rf = accuracy_score(y_test, y_pred_rf)
    print("Random Forest Accuracy:", accuracy_rf)


   

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Random Forest Accuracy: 0.6408450704225352
Random Forest Accuracy: 0.7007042253521126
Random Forest Accuracy: 0.7208480565371025
Random Forest Accuracy: 0.7561837455830389
Random Forest Accuracy: 0.7879858657243817


In [107]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Testing Stratified K Fold instead of K test split
tech_training['combined'] = tech_training['jobtitle'] + tech_training['skills'] + tech_training['jobdescription'] 
X = tech_training['combined'].apply(lambda x:x[0])


y = tech_training['tag'].apply(lambda x: x[0])


skf = StratifiedKFold(n_splits=5)
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    # Initialize 
    tfidf_vectorizer = TfidfVectorizer()

    # Fit transform the vectorizer

    X_train_Tfidf_df = tfidf_vectorizer.fit_transform(X_train).toarray()


    X_test_Tfidf_df = tfidf_vectorizer.transform(X_test).toarray()


    #tfidf_vectorizer.get_feature_names_out()
    X_train_Tfidf_df

    model = CategoricalNB()

    model.fit(X_train_Tfidf_df, y_train)

    y_pred = model.predict(X_test_Tfidf_df)

    accuracy = accuracy_score(y_test, y_pred)

    print("Accuracy: ", accuracy)

    if y_pred.ndim == 1:
        y_pred = pd.Series(y_pred, index=y_test.index)
    else:
        y_pred = pd.DataFrame(y_pred, index=y_test.index)

   


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  warnings.warn(


Accuracy:  0.5070422535211268
Accuracy:  0.49295774647887325
Accuracy:  0.48056537102473496
Accuracy:  0.519434628975265
Accuracy:  0.5159010600706714


In [108]:
model = CategoricalNB()

model.fit(X_train_Tfidf_df, y_train)


CategoricalNB()

In [109]:
y_pred = model.predict(X_test_Tfidf_df)

In [110]:
accuracy_score(y_test, y_pred)


0.5159010600706714

In [111]:
if y_pred.ndim == 1:
    y_pred = pd.Series(y_pred, index=y_test.index)
else:
    y_pred = pd.DataFrame(y_pred, index=y_test.index)

# Create DataFrame for checking
check = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
check



,Actual,Predicted
35,architect,backenddev
36,architect,backenddev
37,architect,backenddev
38,architect,backenddev
39,architect,backenddev
...,...,...
1412,webdev,nontech
1413,webdev,nontech
1414,webdev,nontech
1415,webdev,nontech
